In [1]:
!mkdir ade20k
!wget http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip -P ade20k

--2022-08-16 12:50:54--  http://data.csail.mit.edu/places/ADEchallenge/ADEChallengeData2016.zip
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 967382037 (923M) [application/zip]
Saving to: ‘ade20k/ADEChallengeData2016.zip’

ADEChallengeData201 100%[===================>] 922.57M  12.6MB/s    in 77s     

2022-08-16 12:52:12 (12.0 MB/s) - ‘ade20k/ADEChallengeData2016.zip’ saved [967382037/967382037]



In [2]:
#!wget http://data.csail.mit.edu/places/ADEchallenge/release_test.zip -P ade20k

In [3]:
!pip3 install openmim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 235 kB 60.4 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 


In [4]:
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.6 MB 1.4 MB/s 
     |████████████████████████████████| 190 kB 18.3 MB/s 


In [5]:
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 10481, done.
remote: Counting objects: 100% (356/356), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 10481 (delta 160), reused 290 (delta 130), pack-reused 10125
Receiving objects: 100% (10481/10481), 14.60 MiB | 4.33 MiB/s, done.
Resolving deltas: 100% (7590/7590), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 578 kB 20.8 MB/s 
  Running setup.py develop for mmsegmentation


In [6]:
!mkdir /content/mmsegmentation/data
!mkdir /content/mmsegmentation/data/ade

In [7]:
!unzip -q /content/ade20k/ADEChallengeData2016.zip -d /content/mmsegmentation/data/ade

In [8]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.12.1+cu113 True
0.27.0


In [9]:
!mkdir checkpoints
#!wget https://download.openmmlab.com/mmsegmentation/v0.5/knet/knet_s3_fcn_r50-d8_8x2_512x512_adamw_80k_ade20k/knet_s3_fcn_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_043751-abcab920.pth -P checkpoints
!wget https://download.openmmlab.com/mmsegmentation/v0.5/knet/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth -P checkpoints

--2022-08-16 12:53:08--  https://download.openmmlab.com/mmsegmentation/v0.5/knet/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 161.117.242.73
Connecting to download.openmmlab.com (download.openmmlab.com)|161.117.242.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 326152452 (311M) [application/octet-stream]
Saving to: ‘checkpoints/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth’

knet_s3_deeplabv3_r 100%[===================>] 311.04M  11.4MB/s    in 24s     

2022-08-16 12:53:33 (12.9 MB/s) - ‘checkpoints/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth’ saved [326152452/326152452]



In [10]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [12]:

from mmcv import Config

#config_file = '/content/mmsegmentation/configs/knet/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k.py'

#yukarıdakini yükleyip aşağıdaki gibi kod ile düzenleyince hata aldım ve düzeltemedim. Dosyanın kendisinden aynı değişiklikleri 
#yapıp çalıştırınca oldu.
config_file='/content/mmsegmentation/configs/knet/knet_deep_test.py'
#'configs/knet/knet_s3_fcn_r50-d8_8x2_512x512_adamw_80k_ade20k.py'
checkpoint_file = 'checkpoints/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth'
#'checkpoints/knet_s3_fcn_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_043751-abcab920.pth'
cfg = Config.fromfile(config_file)


In [14]:
'''
from mmseg.apis import set_random_seed
from mmseg.utils import get_device


# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

cfg.runner.max_iters = 200
cfg.log_config.interval = 10
cfg.evaluation.interval = 200
cfg.checkpoint_config.interval = 200

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'



# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()

'''

"\nfrom mmseg.apis import set_random_seed\nfrom mmseg.utils import get_device\n\n\n# Since we use only one GPU, BN is used instead of SyncBN\ncfg.norm_cfg = dict(type='BN', requires_grad=True)\ncfg.model.backbone.norm_cfg = cfg.norm_cfg\ncfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg\n\ncfg.runner.max_iters = 200\ncfg.log_config.interval = 10\ncfg.evaluation.interval = 200\ncfg.checkpoint_config.interval = 200\n\n# We can still use the pre-trained Mask RCNN model though we do not need to\n# use the mask branch\ncfg.load_from = checkpoint_file\n\n# Set up working dir to save files and logs.\ncfg.work_dir = './work_dirs/tutorial'\n\n\n\n# Set seed to facitate reproducing the result\ncfg.seed = 0\nset_random_seed(0, deterministic=False)\ncfg.gpu_ids = range(1)\ncfg.device = get_device()\n\n"

In [15]:
from mmseg.apis import set_random_seed
from mmseg.utils import get_device

cfg.load_from = checkpoint_file
cfg.work_dir = './work_dirs/tutorial'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()


In [16]:
# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'ADE20KDataset'
data_root = 'data/ade/ADEChallengeData2016'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
crop_size = (512, 512)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=True),
    dict(type='Resize', img_scale=(2048, 512), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=(512, 512), cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size=(512, 512), pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(2048, 512),
        flip=False,
        tra

In [17]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
import mmcv
import os.path as osp


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_segmentor(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

2022-08-16 12:54:58,533 - mmseg - INFO - Loaded 20210 images
/content/mmsegmentation/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
2022-08-16 12:55:03,600 - mmseg - INFO - Loaded 2000 images
2022-08-16 12:55:03,602 - mmseg - INFO - load checkpoint from local path: checkpoints/knet_s3_deeplabv3_r50-d8_8x2_512x512_adamw_80k_ade20k_20220228_041642-00c8fbeb.pth
2022-08-16 12:55:03,864 - mmseg - INFO - Start running, host: root@f1632e44cb68, work_dir: /content/mmsegmentation/work_dirs/tutorial
2

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2000/2000, 4.3 task/s, elapsed: 467s, ETA:     0s

2022-08-16 13:08:47,498 - mmseg - INFO - per class results:
2022-08-16 13:08:47,509 - mmseg - INFO - 
+---------------------+-------+-------+
|        Class        |  IoU  |  Acc  |
+---------------------+-------+-------+
|         wall        | 73.82 | 87.08 |
|       building      | 79.12 | 91.29 |
|         sky         | 92.49 | 95.82 |
|        floor        | 77.02 | 87.46 |
|         tree        | 70.45 |  85.9 |
|       ceiling       | 79.77 | 89.16 |
|         road        | 79.52 | 86.49 |
|         bed         | 83.62 | 96.09 |
|      windowpane     | 56.68 | 70.37 |
|        grass        | 60.13 | 80.95 |
|       cabinet       | 54.65 | 67.93 |
|       sidewalk      | 61.09 | 75.09 |
|        person       | 76.15 | 87.52 |
|        earth        |  27.2 | 37.65 |
|         door        | 44.51 | 59.13 |
|        table        |  54.3 | 69.59 |
|       mountain      | 46.54 |  62.5 |
|        plant        | 51.12 | 64.11 |
|       curtain       | 64.33 | 80.75 |
|        chair    

In [19]:
import matplotlib.pyplot as plt
from mmseg.core.evaluation import get_palette
model.cfg = cfg
#img = '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/validation/ADE_val_00001000.jpg'
resimler=['validation/ADE_val_00001962.jpg' , 'validation/ADE_val_00001963.jpg' , 'validation/ADE_val_00001457.jpg' , 
          'validation/ADE_val_00001458.jpg' , 'validation/ADE_val_00001459.jpg' , 'training/ADE_train_00000565.jpg' ,
          'training/ADE_train_00000936.jpg' , 'training/ADE_train_00000952.jpg' , 'training/ADE_train_00003194.jpg' , 
          'training/ADE_train_00006845.jpg']
for img in resimler:
  result = inference_segmentor(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img)
  show_result_pyplot(model, '/content/mmsegmentation/data/ade/ADEChallengeData2016/images/'+img, result,get_palette('ade20k'))